In [ ]:
import pandas as pd
from spark_session import LocalSparkSession
from dataset import Dataset
from mr_id3 import MapReduceIDR3
from pyspark.mllib.tree import DecisionTree

In [ ]:
from log import log
from sklearn.impute import SimpleImputer
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [ ]:
%time
num_fields = [
    'age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
    'hours_per_week', ]

categorical_fields = [
    'workclass', 'education',
    'marital_status', 'occupation', 'relationship',
    'race', 'sex', 'native_country', ]

In [ ]:
%time
target = 'label'
filename = 'dataset/adult.data'

In [ ]:
%time
cores = [20, ]
multiply = list(range(1, 21))
metrics = []

In [ ]:
%time
for n_cores in cores:
    for m_factor in multiply:
        spark = LocalSparkSession(n_cores)
        spark.start()

        dataset = Dataset(spark.spark, filename, num_fields, categorical_fields, target)
        dataset.load()
        dataset.one_hot_encode_categorical_fields()
        dataset.multiply_dataset(m_factor)

        mr_id3 = MapReduceIDR3(dataset)
        mr_id3.set_labeled_point()
        mr_id3.split()
        mr_id3.train()

        metric = mr_id3.get_metrics()
        metric['length_rows'] = dataset.df.count()
        metric['m_factor'] = m_factor
        metric['n_cores'] = n_cores
        metrics.append(metric)
        log(f"Metrics: Clusters {metric['n_cores']} - File size {metric['m_factor']}x - Time {metric['time']} seconds")

        spark.stop()

In [ ]:
%time
df = pd.DataFrame.from_dict(metrics)

In [ ]:
df

In [ ]:
%time
df.plot.line('m_factor', 'time')